# Test Driven Development

**Project Goals**: 
- Take the code for accessing the API and build an AlphaVantageAPI class.
- Create a SQLRepository class that will help us load our stock into a SQLite database, then extract then extract it for later use.

In [3]:
%load_ext autoreload
%load_ext sql
%autoreload 2

import sqlite3

import matplotlib.pyplot as plt
import pandas as pd
from config import settings

The sql module is not an IPython extension.


### AlphaVantageAPI Class

- Importing the ```AlphaVantageAPI``` class from the data module and create an instance of it 

In [4]:
# Import `AlphaVantageAPI`
from data import AlphaVantageAPI

# Create instance of `AlphaVantageAPI` class
av = AlphaVantageAPI()

print("av type:", type(av))

av type: <class 'data.AlphaVantageAPI'>


### AlphaVantageAPI: Get_daily Method

- Using the ```get_daily``` method to fetch the  stock data for the renewable energy company Suzlon and assign it to the DataFrame df_suzlon.

In [5]:
# Define Suzlon ticker symbol
ticker = "SUZLON.BSE"

# Use your `av` object to get daily data
df_suzlon = av.get_daily(ticker)

print("df_suzlon type:", type(df_suzlon))
print("df_suzlon shape:", df_suzlon.shape)
df_suzlon.head()

df_suzlon type: <class 'pandas.core.frame.DataFrame'>
df_suzlon shape: (4589, 5)


,open,high,low,close,volume
date,,,,,
2024-06-05,45.13,49.20,45.13,48.35,14381150.0
2024-06-04,52.19,52.19,47.50,47.50,8835466.0
2024-06-03,49.99,49.99,49.99,49.99,2611005.0
2024-05-31,45.40,47.62,44.21,47.61,4181822.0
2024-05-30,46.40,46.90,44.70,45.36,6019019.0


### AlphaVantageAPI: Testing Get_daily Method

- assert statements to test the output of your ```get_daily``` method

In [6]:
# Does `get_daily` return a DataFrame?
assert isinstance(df_suzlon, pd.DataFrame)

# Does DataFrame have 5 columns?
assert df_suzlon.shape[1] == 5

# Does DataFrame have a DatetimeIndex?
assert isinstance(df_suzlon.index , pd.DatetimeIndex)

# Is the index name "date"?
assert df_suzlon.index.name  == "date"

More Tests:

In [7]:
# Does DataFrame have correct column names?
assert all(df_suzlon.columns == ['open', 'high', 'low', 'close', 'volume'])
assert df_suzlon.columns.to_list() == ['open', 'high', 'low', 'close', 'volume']
# Are columns correct data type?
assert all(df_suzlon.dtypes == float)

### SQL Repository Class

- For storing data into the database. Because our data is highly structured (each DataFrame we extract from AlphaVantage is always going to have the same five columns), it makes sense to use a SQL database.

#### Connect to Database

In [8]:
connection = sqlite3.connect(database=settings.db_name, check_same_thread=False)

print("connection type:", type(connection))

connection type: <class 'sqlite3.Connection'>


#### SQLRepository: Tests

In [9]:
# Import class definition
from data import SQLRepository

# Create instance of class
repo = SQLRepository(connection=connection)

# Does `repo` have a "connection" attribute?
hasattr (repo, "connection") 

# # Is the "connection" attribute a SQLite `Connection`?
assert isinstance(repo.connection, sqlite3.Connection)

#### SQLRepository: Tests for Insert_table Method

In [10]:
response = repo.insert_table(table_name=ticker, records=df_suzlon, if_exists="replace")

# Does your method return a dictionary?
assert isinstance(response, dict)

# Are the keys of that dictionary correct?
assert sorted(list(response.keys())) == ["records_inserted", "transaction_successful"]

In [11]:
response

{'transaction_successful': True, 'records_inserted': 4589}